<a href="https://colab.research.google.com/github/ganeshIIT/airbnb/blob/master/airbnb.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install pyodbc -q

!curl https://packages.microsoft.com/keys/microsoft.asc | apt-key add -
!curl https://packages.microsoft.com/config/ubuntu/16.04/prod.list > /etc/apt/sources.list.d/mssql-release.list
!sudo apt-get update
!sudo ACCEPT_EULA=Y apt-get -q -y install msodbcsql17

In [58]:
import numpy as np
import pandas as pd
import pyodbc

import dataloader

In [3]:
from datetime import datetime

from bs4 import BeautifulSoup
from urllib.request import Request, urlopen
import re

import helper

req = Request("http://insideairbnb.com/get-the-data/")
html_page = urlopen(req)

soup = BeautifulSoup(html_page, "lxml")

links = []
for link in soup.findAll('a'):
    links.append(link.get('href'))

# print(links)


In [4]:
def getdetailsfromurl(urltext):
    components = urltext.split('/')
    country = components[3]
    if len(components) == 9:
        province = components[4]
        city = components[5]
    else:
        province = components[3]
        city = components[3]
    return {'country': country, 'province':province, 'city': city}


# print(getdetailsfromurl('http://data.insideairbnb.com/new-zealand/2022-09-12/data/listings.csv.gz'))
# getdetailsfromurl('http://data.insideairbnb.com/united-states/dc/washington-dc/2022-09-14/data/listings.csv.gz')

In [5]:
data = {}
id = 0
for link in links:
        if link:
            if '.csv.gz' in link:
                data[id]= getdetailsfromurl(link)
                id+=1
                
data = pd.DataFrame(data).T
data['country'].drop_duplicates().to_frame().reset_index().drop('index', axis =1)
data['province'].drop_duplicates().to_frame().reset_index().drop('index', axis =1)
data['city'].drop_duplicates().to_frame().reset_index().drop('index', axis =1)

,city
0,amsterdam
1,antwerp
2,asheville
3,athens
4,austin
...,...
110,winnipeg
111,zurich
112,ireland
113,malta


In [6]:
def getdatalinks(datasets = ['listings', 'calendar', 'reviews'], 
                 country = 'united-states', 
                 province = None, 
                 city = None):
    
    url_dict = {name:list() for name in datasets}
    for link in links:
        if link:
            if (not province) and (not city):
                for dataset in datasets:
                    if f'/{dataset}.csv.gz' in link and f'/{country}/' in link:
                        url_dict[dataset].append(link)
          
            if (province) and (not city):
                for dataset in datasets:
                    if f'/{dataset}.csv.gz' in link and f'/{country}/{province}/' in link:
                        url_dict[dataset].append(link)

            if province and city:
                for dataset in datasets:
                    if f'/{dataset}.csv.gz' in link and f'/{country}/{province}/{city}/' in link:
                        url_dict[dataset].append(link)
    return(url_dict)

In [7]:
country = 'united-states'
province = 'nc'
city = 'asheville'
datasets = ['listings', 'calendar', 'reviews']

getdatalinks(datasets = datasets, country = country, province=province, city= city)

{'listings': ['http://data.insideairbnb.com/united-states/nc/asheville/2022-09-14/data/listings.csv.gz'],
 'calendar': ['http://data.insideairbnb.com/united-states/nc/asheville/2022-09-14/data/calendar.csv.gz'],
 'reviews': ['http://data.insideairbnb.com/united-states/nc/asheville/2022-09-14/data/reviews.csv.gz']}

In [8]:
urls = getdatalinks(datasets = datasets, 
                    country = country, 
                    # province=province, 
                    # city= city
                    )
urls

{'listings': ['http://data.insideairbnb.com/united-states/nc/asheville/2022-09-14/data/listings.csv.gz',
  'http://data.insideairbnb.com/united-states/tx/austin/2022-09-12/data/listings.csv.gz',
  'http://data.insideairbnb.com/united-states/ma/boston/2022-06-13/data/listings.csv.gz',
  'http://data.insideairbnb.com/united-states/fl/broward-county/2022-06-17/data/listings.csv.gz',
  'http://data.insideairbnb.com/united-states/ma/cambridge/2022-06-22/data/listings.csv.gz',
  'http://data.insideairbnb.com/united-states/il/chicago/2022-09-14/data/listings.csv.gz',
  'http://data.insideairbnb.com/united-states/nv/clark-county-nv/2022-06-13/data/listings.csv.gz',
  'http://data.insideairbnb.com/united-states/oh/columbus/2022-06-20/data/listings.csv.gz',
  'http://data.insideairbnb.com/united-states/tx/dallas/2022-09-14/data/listings.csv.gz',
  'http://data.insideairbnb.com/united-states/co/denver/2022-09-26/data/listings.csv.gz',
  'http://data.insideairbnb.com/united-states/tx/fort-worth/20

In [49]:
listing_columns = ['id', 'listing_url', 'scrape_id', 'last_scraped', 'source', 'name',
       'neighborhood_overview', 'picture_url', 'host_id',
       'host_url', 'host_name', 'host_since', 'host_location', 'host_about',
       'host_response_time', 'host_response_rate', 'host_acceptance_rate',
       'host_is_superhost', 'host_thumbnail_url', 'host_picture_url',
       'host_neighbourhood', 'host_listings_count',
       'host_total_listings_count', 'host_verifications',
       'host_has_profile_pic', 'host_identity_verified', 'neighbourhood',
       'neighbourhood_cleansed', 'neighbourhood_group_cleansed', 'latitude',
       'longitude', 'property_type', 'room_type', 'accommodates', 'bathrooms',
       'bathrooms_text', 'bedrooms', 'beds', 'amenities', 'price',
       'minimum_nights', 'calendar_updated', 'has_availability',
       'availability_30', 'availability_60', 'availability_90',
       'availability_365', 'calendar_last_scraped', 'number_of_reviews',
       'number_of_reviews_ltm', 'number_of_reviews_l30d', 'first_review',
       'last_review', 'review_scores_rating', 'review_scores_accuracy',
       'review_scores_cleanliness', 'review_scores_checkin',
       'review_scores_communication', 'review_scores_location',
       'review_scores_value', 'license', 'instant_bookable',
       'calculated_host_listings_count',
       'calculated_host_listings_count_entire_homes',
       'calculated_host_listings_count_private_rooms',
       'calculated_host_listings_count_shared_rooms', 'reviews_per_month']

calendar_columns = ['listing_id', 
                    'date', 
                    'available', 
                    'price', 
                     #'adjusted_price',
                     # 'minimum_nights', 'maximum_nights'
                     ]

reviews_columns = ['listing_id', 
                   'id', 
                   'date', 
                   'reviewer_id', 
                   'reviewer_name', 
                   'comments']

In [12]:
all_listings = pd.DataFrame()
all_calendar = pd.DataFrame()
all_reviews = pd.DataFrame()

for url in urls['listings']:
    # match = re.search('\d{4}-\d{2}-\d{2}', url)
    # print(datetime.strptime(match.group(), '%Y-%m-%d').date())
    #print(getdetailsfromurl(url))
    listings = (pd.read_csv(url)
            .assign(country = getdetailsfromurl(url)['country'],
                    province = getdetailsfromurl(url)['province'],
                    city = getdetailsfromurl(url)['city'])
    )
    all_listings = pd.concat([all_listings, listings])
all_listings = all_listings[listing_columns]
    
for url in urls['calendar']:
    calendar = pd.read_csv(url, parse_dates=['date'])
    calendar = calendar.loc[:,calendar_columns].loc[calendar['date'].isin(pd.date_range(start =datetime.now().date(), periods=10, freq='D'))]
    all_calendar = pd.concat([all_calendar, calendar])
    
for url in urls['reviews']:
    reviews =pd.read_csv(url, parse_dates=['date'])
    reviews = reviews.sort_values(by = 'date', ascending=False).groupby('listing_id').head(10)
    all_reviews = pd.concat([all_reviews, reviews])
all_reviews = all_reviews[reviews_columns]

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:3326: DtypeWarning: Columns (68) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [14]:
print(all_listings.shape)
print(all_calendar.shape)
print(all_reviews.shape)

(275303, 74)
(2752070, 4)
(1704944, 6)


In [22]:
all_listings.drop_duplicates(subset = ['id', 'listing_url'], inplace = True)

In [50]:
all_listings = all_listings[listing_columns]

In [44]:
all_listings=(
        all_listings.rename(columns = {'id':'listing_id'})
            .assign(listing_id = lambda x: x['listing_id'].astype('object'),
                    scrape_id = lambda x: x['scrape_id'].astype('object'),
                    host_id = lambda x: x['host_id'].astype('object'),
        )
    )

In [51]:
all_listings.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 270517 entries, 0 to 6472
Data columns (total 67 columns):
 #   Column                                        Non-Null Count   Dtype  
---  ------                                        --------------   -----  
 0   listing_id                                    270517 non-null  object 
 1   listing_url                                   270517 non-null  object 
 2   scrape_id                                     270517 non-null  object 
 3   last_scraped                                  270517 non-null  object 
 4   source                                        208829 non-null  object 
 5   name                                          270500 non-null  object 
 6   neighborhood_overview                         167961 non-null  object 
 7   picture_url                                   270516 non-null  object 
 8   host_id                                       270517 non-null  object 
 9   host_url                                      2705

In [ ]:
all_listings[all_listings['id']>2147483000]

In [45]:
all_listings.describe().T

,count,mean,std,min,25%,50%,75%,max
host_listings_count,269306.0,1.060277e+02,4.992277e+02,0.00000,1.00000,3.00000,16.00000,4.129000e+03
host_total_listings_count,269306.0,1.441007e+02,6.874253e+02,0.00000,1.00000,4.00000,20.00000,2.000000e+04
latitude,270517.0,3.468001e+01,6.716393e+00,18.92025,30.50338,34.19168,40.67954,4.774800e+01
longitude,270517.0,-1.049503e+02,2.553452e+01,-159.71462,-118.43447,-104.99664,-80.13278,-7.091838e+01
accommodates,270517.0,4.297815e+00,2.925442e+00,0.00000,2.00000,4.00000,6.00000,1.600000e+01
bathrooms,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
bedrooms,247722.0,1.829345e+00,1.178079e+00,1.00000,1.00000,1.00000,2.00000,2.400000e+01
beds,266379.0,2.397445e+00,1.946523e+00,1.00000,1.00000,2.00000,3.00000,1.320000e+02
minimum_nights,270517.0,1.271898e+01,2.837577e+01,1.00000,2.00000,3.00000,30.00000,1.250000e+03
maximum_nights,270517.0,8.685380e+03,4.129285e+06,1.00000,60.00000,365.00000,1125.00000,2.147484e+09


In [ ]:
dataloader.full_load(df = all_listings, tbl = "listings", hasindex = False)